In [4]:
import os
os.environ['KAGGLE_USERNAME'] = 'abhinavpandey1729'  
os.environ['KAGGLE_KEY'] = '6ca9b44f97cb00217533ef28f8859223'          

In [3]:
import kagglehub
kagglehub.auth.login()

In [ ]:
# Download the dataset
path = kagglehub.dataset_download("yanmaksi/big-molecules-smiles-dataset")
print("Path to dataset files:", path)

Path to dataset files: /Users/lykofos/.cache/kagglehub/datasets/yanmaksi/big-molecules-smiles-dataset/versions/1


In [5]:
from sklearn.preprocessing import MinMaxScaler

Python(61164) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [6]:
import numpy as np
import pandas as pd

In [8]:
dataset_path = path


In [9]:
csv_file =  os.path.join(dataset_path, "SMILES_Big_Data_Set.csv")

In [10]:
df = pd.read_csv(csv_file)

In [11]:
df.head()

,SMILES,pIC50,mol,num_atoms,logP
0,O=S(=O)(Nc1cccc(-c2cnc3ccccc3n2)c1)c1cccs1,4.26,<rdkit.Chem.rdchem.Mol object at 0x7f59df45bc30>,25,4.1591
1,O=c1cc(-c2nc(-c3ccc(-c4cn(CCP(=O)(O)O)nn4)cc3)...,4.34,<rdkit.Chem.rdchem.Mol object at 0x7f59a320c9e0>,36,3.6743
2,NC(=O)c1ccc2c(c1)nc(C1CCC(O)CC1)n2CCCO,4.53,<rdkit.Chem.rdchem.Mol object at 0x7f59a320cac0>,23,1.5361
3,NCCCn1c(C2CCNCC2)nc2cc(C(N)=O)ccc21,4.56,<rdkit.Chem.rdchem.Mol object at 0x7f59a320cba0>,22,0.9510
4,CNC(=S)Nc1cccc(-c2cnc3ccccc3n2)c1,4.59,<rdkit.Chem.rdchem.Mol object at 0x7f59a320c7b0>,21,3.2130


In [12]:
df['SMILES'].value_counts()

SMILES
CC(NC1CCC(n2cc(C(N)=O)c(Nc3ccc(S(=O)(=O)C(F)(F)F)cc3)n2)C(C#N)C1)C1CC1    6
N#CC1CC(N2CCC2)CCC1n1cc(C(N)=O)c(Nc2ccc(C(O)C(F)(F)F)cc2)n1               4
CC1(C)CN(C2CCC(n3cc(C(N)=O)c(Nc4ccc(C(C)(O)C(F)(F)F)cc4)n3)C(C#N)C2)C1    4
CC(O)(c1ccc(Nc2nn(C3CCC(N4CCC4)CC3C#N)cc2C(N)=O)cc1)C(F)(F)F              4
CC1(C)CN(C2CCC(n3cc(C(N)=O)c(Nc4ccc(C(O)C(F)(F)F)cc4)n3)C(C#N)C2)C1       4
                                                                         ..
COC(=O)c1ccncc1                                                           1
COc1cc(Cl)c(Cl)cc1O                                                       1
COP(=S)(OC)Oc1ccc([N+](=O)[O-])cc1Cl                                      1
CC(=O)n1ccnc1                                                             1
N=[N+]=NC=Cc1ccc([N+](=O)[O-])o1                                          1
Name: count, Length: 15872, dtype: int64

In [13]:
df.duplicated().value_counts() #duplicated values

False    16087
Name: count, dtype: int64

In [14]:
df.drop_duplicates(inplace=True)

In [15]:
from rdkit import Chem
from rdkit.Chem import Descriptors


In [16]:
# Function to get molecular features
def get_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return {
        'MolecularWeight': Descriptors.MolWt(mol),  # For tempo
        'NumAtoms': mol.GetNumAtoms(),             # For pitch
        'NumBonds': mol.GetNumBonds()             # For rhythm
    }

# Add features to dataset
features = df['SMILES'].apply(get_features).apply(pd.Series)
df = pd.concat([df[['SMILES']], features], axis=1)
print("Added features:", list(features.columns))

Added features: ['MolecularWeight', 'NumAtoms', 'NumBonds']


In [17]:
# Remove rows with missing values
df = df.dropna().reset_index(drop=True)
print("After cleaning, kept", len(df), "rows")

After cleaning, kept 16087 rows


In [19]:
# Normalize features to 0-1
scaler = MinMaxScaler()
feature_columns = ['MolecularWeight', 'NumAtoms', 'NumBonds']
df[feature_columns] = scaler.fit_transform(df[feature_columns])

In [20]:
# Save to CSV
df.to_csv("preprocessed_molecules.csv", index=False)

In [21]:
df.head()

,SMILES,MolecularWeight,NumAtoms,NumBonds
0,O=S(=O)(Nc1cccc(-c2cnc3ccccc3n2)c1)c1cccs1,0.281609,0.268293,0.302326
1,O=c1cc(-c2nc(-c3ccc(-c4cn(CCP(=O)(O)O)nn4)cc3)...,0.401153,0.402439,0.441860
2,NC(=O)c1ccc2c(c1)nc(C1CCC(O)CC1)n2CCCO,0.238544,0.243902,0.267442
3,NCCCn1c(C2CCNCC2)nc2cc(C(N)=O)ccc21,0.224786,0.231707,0.255814
4,CNC(=S)Nc1cccc(-c2cnc3ccccc3n2)c1,0.218755,0.219512,0.244186


In [22]:
def get_features(smiles):
    """Get simple molecular features from SMILES."""
    mol = Chem.MolFromSmiles(smiles)
    return {
        'MolecularWeight': Descriptors.MolWt(mol),  # Weight in Daltons
        'NumAtoms': mol.GetNumAtoms(),             # Total atoms
        'NumBonds': mol.GetNumBonds()             # Total bonds
    }

In [23]:
# Apply feature extraction
features = df['SMILES'].apply(get_features).apply(pd.Series)
df = pd.concat([df[['SMILES']], features], axis=1)
print("Extracted features:", list(features.columns))

Extracted features: ['MolecularWeight', 'NumAtoms', 'NumBonds']


In [24]:
#Define audio mappings
def map_to_audio(row):
    """Map molecular features to audio parameters."""
    return {
        'Tempo': 60 + (row['MolecularWeight'] / 500) * 120,  # Tempo: 60-180 BPM
        'Pitch': 200 + (row['NumAtoms'] / 50) * 1000,       # Pitch: 200-1200 Hz
        'RhythmComplexity': row['NumBonds'] / 50            # Rhythm: 0-1 scale
    }

In [25]:
audio_features = df.apply(map_to_audio, axis=1).apply(pd.Series)
df = pd.concat([df, audio_features], axis=1)
print( list(audio_features.columns))

['Tempo', 'Pitch', 'RhythmComplexity']


In [26]:
correlation_matrix = df[['MolecularWeight', 'NumAtoms', 'NumBonds', 'Tempo', 'Pitch', 'RhythmComplexity']].corr()
print("\nCorrelation Matrix:")
print(correlation_matrix)


Correlation Matrix:
                  MolecularWeight  NumAtoms  NumBonds     Tempo     Pitch  \
MolecularWeight          1.000000  0.962643   0.95384  1.000000  0.962643   
NumAtoms                 0.962643  1.000000   0.99590  0.962643  1.000000   
NumBonds                 0.953840  0.995900   1.00000  0.953840  0.995900   
Tempo                    1.000000  0.962643   0.95384  1.000000  0.962643   
Pitch                    0.962643  1.000000   0.99590  0.962643  1.000000   
RhythmComplexity         0.953840  0.995900   1.00000  0.953840  0.995900   

                  RhythmComplexity  
MolecularWeight            0.95384  
NumAtoms                   0.99590  
NumBonds                   1.00000  
Tempo                      0.95384  
Pitch                      0.99590  
RhythmComplexity           1.00000  


## This shows how our molecule numbers relate to music numbers, helping us understand if our mappings make sense.”
## We Use df.corr() to generate a correlation matrix, where values near 1 or -1 indicate strong relationships.

In [27]:
# Save updated dataset with features and audio mappings
df.to_csv("molecule_audio_features.csv", index=False)
print("Saved dataset with features and audio mappings to molecule_audio_features.csv")

# Preview the data
print("\nPreview of data with features and audio mappings:")
print(df[['SMILES', 'MolecularWeight', 'NumAtoms', 'NumBonds', 'Tempo', 'Pitch', 'RhythmComplexity']].head())

Saved dataset with features and audio mappings to molecule_audio_features.csv

Preview of data with features and audio mappings:
                                              SMILES  MolecularWeight  \
0         O=S(=O)(Nc1cccc(-c2cnc3ccccc3n2)c1)c1cccs1          367.455   
1  O=c1cc(-c2nc(-c3ccc(-c4cn(CCP(=O)(O)O)nn4)cc3)...          506.434   
2             NC(=O)c1ccc2c(c1)nc(C1CCC(O)CC1)n2CCCO          317.389   
3                NCCCn1c(C2CCNCC2)nc2cc(C(N)=O)ccc21          301.394   
4                  CNC(=S)Nc1cccc(-c2cnc3ccccc3n2)c1          294.383   

   NumAtoms  NumBonds      Tempo  Pitch  RhythmComplexity  
0      25.0      28.0  148.18920  700.0              0.56  
1      36.0      40.0  181.54416  920.0              0.80  
2      23.0      25.0  136.17336  660.0              0.50  
3      22.0      24.0  132.33456  640.0              0.48  
4      21.0      23.0  130.65192  620.0              0.46  
